In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'sampleSubmission.csv', 'train.csv']


# 1. Data EDA

Data EDA와 Feature Engineering은 우선 저번 커널에서 했던 것과 거의 비슷하게 진행했다. 

In [3]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [4]:
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [5]:
test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


In [6]:
train = pd.read_csv("../input/train.csv", parse_dates = ["datetime"])
test = pd.read_csv("../input/test.csv", parse_dates = ["datetime"])

In [7]:
train["year"] = train["datetime"].dt.year
train["hour"] = train["datetime"].dt.hour
train["dayofweek"] = train["datetime"].dt.dayofweek

test["year"] = test["datetime"].dt.year
test["hour"] = test["datetime"].dt.hour
test["dayofweek"] = test["datetime"].dt.dayofweek

In [12]:
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,hour,dayofweek
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,0,5
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,5
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,2,5
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011,3,5
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011,4,5


In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 15 columns):
datetime      10886 non-null datetime64[ns]
season        10886 non-null int64
holiday       10886 non-null int64
workingday    10886 non-null int64
weather       10886 non-null int64
temp          10886 non-null float64
atemp         10886 non-null float64
humidity      10886 non-null int64
windspeed     10886 non-null float64
casual        10886 non-null int64
registered    10886 non-null int64
count         10886 non-null int64
year          10886 non-null int64
hour          10886 non-null int64
dayofweek     10886 non-null int64
dtypes: datetime64[ns](1), float64(3), int64(11)
memory usage: 1.2 MB


여기부터 저번과 조금 다르게 해보려고 한다. count는 대여량 총량이지만 casual은 일반 사용자들, registered는 자전거를 등록하고 사용하는 사용자들이라고 생각되므로 두개를 예측을 따로 해서 count를 더해봐야겠다.

In [14]:
y_train_cas = train["casual"]

In [15]:
y_train_cas.head()

0    3
1    8
2    5
3    3
4    0
Name: casual, dtype: int64

In [16]:
y_train_reg = train["registered"]

In [18]:
y_train_reg.head()

0    13
1    32
2    27
3    10
4     1
Name: registered, dtype: int64

두 y를 저번처럼 로그화를 해주자.

In [19]:
y_train_cas = np.log1p(y_train_cas)

In [20]:
y_train_reg = np.log1p(y_train_reg)

In [21]:
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,hour,dayofweek
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,0,5
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,5
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,2,5
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011,3,5
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011,4,5


이제 학습시킬 x에서 필요없는 컬럼을 저번처럼 드랍해주자.

In [22]:
train.drop(["datetime", "windspeed",  "casual", "registered", "count"], 1, inplace=True)

In [23]:
train.head()

,season,holiday,workingday,weather,temp,atemp,humidity,year,hour,dayofweek
0,1,0,0,1,9.84,14.395,81,2011,0,5
1,1,0,0,1,9.02,13.635,80,2011,1,5
2,1,0,0,1,9.02,13.635,80,2011,2,5
3,1,0,0,1,9.84,14.395,75,2011,3,5
4,1,0,0,1,9.84,14.395,75,2011,4,5


test도 마찬가지로 지워준다.

In [24]:
test.drop(["datetime", "windspeed"], 1, inplace=True)
test.head()

,season,holiday,workingday,weather,temp,atemp,humidity,year,hour,dayofweek
0,1,0,1,1,10.66,11.365,56,2011,0,3
1,1,0,1,1,10.66,13.635,56,2011,1,3
2,1,0,1,1,10.66,13.635,56,2011,2,3
3,1,0,1,1,10.66,12.880,56,2011,3,3
4,1,0,1,1,10.66,12.880,56,2011,4,3


# 2. 모델링

In [25]:
from sklearn.ensemble import RandomForestRegressor

우선 casual 컬럼에 대한 예측값을 먼저 모델을 학습하고 fit해보자.

In [27]:
rf_cas = RandomForestRegressor(n_estimators=100)

In [28]:
rf_cas.fit(train,y_train_cas)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [31]:
preds_cas = rf_cas.predict(test)

registered에 대한 예측값도 모델을 학습하고 fit 해본다.

In [32]:
rf_reg = RandomForestRegressor(n_estimators=100)

In [33]:
rf_reg.fit(train,y_train_reg)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [34]:
preds_reg = rf_reg.predict(test)

예측값이 어떻게 나왔는지 살펴본다.

In [36]:
preds_cas

array([0.80006548, 0.32109042, 0.69903633, ..., 1.39493618, 1.27124745,
       0.99696539])

In [37]:
preds_reg

array([2.45930949, 1.68846071, 0.99657175, ..., 4.60535645, 4.53882746,
       3.7749546 ])

로그화를 먼저 돌려준다.

In [38]:
preds_cas = np.expm1(preds_cas)

In [39]:
preds_reg = np.expm1(preds_reg)

In [40]:
preds_cas

array([1.22568666, 0.37863024, 1.01181305, ..., 3.03471708, 2.56529732,
       1.71004542])

In [41]:
preds_reg

array([10.69673199,  4.41114495,  1.70897883, ..., 99.01862835,
       92.58100815, 42.59552897])

두 예측값을 합쳐주자.

In [42]:
preds = preds_cas + preds_reg

In [43]:
preds

array([ 11.92241865,   4.78977518,   2.72079189, ..., 102.05334543,
        95.14630547,  44.30557439])

제출할 파일을 만든다.

In [44]:
submission=pd.read_csv("../input/sampleSubmission.csv")

In [45]:
submission["count"] = preds 

In [46]:
submission.head()

,datetime,count
0,2011-01-20 00:00:00,11.922419
1,2011-01-20 01:00:00,4.789775
2,2011-01-20 02:00:00,2.720792
3,2011-01-20 03:00:00,2.310768
4,2011-01-20 04:00:00,1.970750


In [48]:
submission.to_csv("rf2.csv", index=False)

# 3. Results

- public score :  0.38656
- rank : 144/3251 (4.43%)

다음 커널에는 부스팅모델을 사용해서 성적을 좀더 올려보겠다!